# 04_04: Smoothing and plotting time series

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
import getweather

def fillnans(array):
    x = np.arange(len(array))
    good = ~np.isnan(array)

    return np.interp(x, x[good], array[good])

In [ ]:
hilo = getweather.getyear('HILO', ['TMIN','TMAX'], 2000)

In [ ]:
hilo['TMIN'], hilo['TMAX'] = fillnans(hilo['TMIN']), fillnans(hilo['TMAX'])

In [ ]:
np.mean(hilo['TMIN']), np.min(hilo['TMIN']), np.max(hilo['TMIN'])

In [ ]:
pp.plot(hilo['TMIN'])

# we loop over a list of functions!
for summary in [np.mean, np.min, np.max]:
    pp.axhline(summary(hilo['TMIN']), linestyle=':') # dotted line

In [ ]:
mean, err = np.mean(hilo['TMIN']), math.sqrt(np.var(hilo['TMIN']))

In [ ]:
pp.plot(hilo['TMIN'])

for value in [mean, mean - err, mean + err]:
    pp.axhline(value, linestyle=':')

In [ ]:
rain = fillnans(getweather.getyear('HILO', ['PRCP'], 2000)['PRCP'])

In [ ]:
pp.plot(rain)

In [ ]:
mask = np.ones(10) / 10

In [ ]:
pp.plot(hilo['TMIN'], '.', ms=3) # smaller dots
pp.plot(range(5, 361), np.convolve(hilo['TMIN'], mask, mode='valid')) # align shorter vector

In [ ]:
pp.plot(hilo['TMIN'], '.', ms=3)
pp.plot(np.convolve(hilo['TMIN'], mask, mode='same'))

In [ ]:
def smooth(array, window=10, mode='valid'):
    return np.convolve(array, np.ones(window)/window, mode)

In [ ]:
len(smooth(hilo['TMIN'], 10))

In [ ]:
pp.plot(hilo['TMIN'], '.', ms=3)
pp.plot(range(5, 361), smooth(hilo['TMIN'], 10))

pp.plot(hilo['TMAX'], '.', ms=3)
pp.plot(range(5, 361), smooth(hilo['TMAX'], 10))

In [ ]:
def plotsmoothed(station, year):
    # get the data
    stationdata = getweather.getyear(station, ['TMIN','TMAX'], year)

    # repeat for TMIN and TMAX
    for obs in ['TMIN','TMAX']:   
        # fill NaNs and smooth
        data = smooth(fillnans(stationdata[obs]), 20)
        # plot, setting x to the post-smoothing range
        pp.plot(range(10, 356), data)

    # add title and set axes limits
    pp.title(station)
    pp.axis(xmin=1, xmax=365, ymin=-10, ymax=45)

In [ ]:
for year in range(2000, 2024):
    plotsmoothed('HILO', year)

pp.axis(ymin=15, ymax=33)

In [ ]:
for i, city in enumerate(['PASADENA', 'NEW YORK', 'SAN DIEGO', 'MINNEAPOLIS'], start=1):
    pp.subplot(2, 2, i)
    plotsmoothed(city, 2023)

pp.tight_layout() # space out plots correctly